In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch  

In [2]:
print(torch.__version__)

1.9.1


In [3]:
data = pd.read_csv('triplets_all.csv')
N = 878
J = torch.zeros((N, N))
for i in range(data.shape[0]):
    a, b = map(int, data.ids[i].split('_'))
    c = int(data.intersize[i])
    if a != b:
        J[a][b] = c
        J[b][a] = c
print(J)

tensor([[0., 5., 2.,  ..., 0., 0., 0.],
        [5., 0., 8.,  ..., 0., 0., 0.],
        [2., 8., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 3.],
        [0., 0., 0.,  ..., 0., 3., 0.]])


In [4]:
p = torch.zeros((N, N))
for i in range(N):
    for j in range(N):
        if i != j:
            p[i][j] = 1

In [5]:
import torch
from torch.nn import Module, Linear
from torch.nn.functional import linear


def Binarize(tensor,quant_mode='det'):
    if quant_mode=='det':
        return tensor.sign()
    if quant_mode=='bin':
        return (tensor>=0).type(type(tensor))*2-1
    else:
        return tensor.add_(1).div_(2).add_(torch.rand(tensor.size()).add(-0.5)).clamp_(0,1).round().mul_(2).add_(-1)


class BNNLinear(Linear):
    def __init__(self, *kargs, **kwargs):
        super(BNNLinear, self).__init__(*kargs, **kwargs)
        self.register_buffer('weight_org', self.weight.data.clone())

    def forward(self, input):
        input.data=Binarize(input.data)
            
        self.weight.data=Binarize(self.weight_org)
        out = linear(input, self.weight)

        out[out == -1] = 0

        # if not self.bias is None:
        #     self.bias.org=self.bias.data.clone()
        #     out += self.bias.view(1, -1).expand_as(out.reshape(1, 16)).reshape(16)

        return out

In [6]:
import torch.nn as nn
import torch

class BNNCaffenet(nn.Module):

    def __init__(self):
        super(BNNCaffenet, self).__init__()
 
        self.features = nn.Sequential(
            # nn.BatchNorm1d(1),
            # nn.Hardtanh(inplace=True),
            BNNLinear(1, 878 * 10000),
        )

    def forward(self, x):
        return self.features(x)


    def init_w(self):
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)
        return

def bnn_caffenet():
    return BNNCaffenet()

In [7]:
import os
import numpy as np
from torch import save, no_grad
from torch import tensor
import torch
from tqdm import tqdm
import shutil


class BnnClassifier():
    def __init__(self, model, device=None):
        super().__init__()
        self.model = model
        self.device = device

    def train_step(self, criterion, optimizer):
        losses = []
        data = torch.ones(1)
        target = J.type(torch.LongTensor)
        mn = 100000000
        for i in tqdm(range(100000), total=100000):
            output = self.model(data)
            loss = criterion(output, target)
            if loss < mn:
                print(loss)
#                 print(output)
            mn = min(mn, loss)
            losses.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            for p in self.model.modules():
                if hasattr(p, 'weight_org'):
                    p.weight.data.copy_(p.weight_org)
            optimizer.step()
            for p in self.model.modules():
                if hasattr(p, 'weight_org'):
                    p.weight_org.data.copy_(p.weight.data.clamp_(-1,1))
        return losses

    def train(self, criterion, optimizer, epochs, scheduler):

        best_accuracy = 0.

        losses = []
        accuracies = []

        self.model.train()
        epoch_losses = self.train_step(criterion, optimizer)
        losses += epoch_losses
        epoch_losses = np.array(epoch_losses)
        lr = optimizer.param_groups[0]['lr']  
        if scheduler:     
            scheduler.step()
        

        return

In [8]:
import torch
import importlib

def myCustomLoss(my_outputs, my_labels):
    a = torch.reshape(my_outputs, (878, 10000))
    tmp = (a @ a.T - my_labels) * p
    return abs(tmp).sum()

device = torch.device('cpu')
torch.manual_seed(0)

model = BNNCaffenet()
model.to(device)

classification = BnnClassifier(model, device)

criterion = myCustomLoss

if hasattr(model, 'init_w'):
    model.init_w()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [80, 150],
        gamma=0.1)

classification.train(criterion, optimizer, 300, scheduler)

  0%|          | 99/100000 [00:44<12:00:20,  2.31it/s]

tensor(98592448., grad_fn=<SumBackward0>)


  0%|          | 100/100000 [00:45<12:02:09,  2.31it/s]

tensor(94588896., grad_fn=<SumBackward0>)


  0%|          | 101/100000 [00:45<12:00:02,  2.31it/s]

...

tensor(589904., grad_fn=<SumBackward0>)


 53%|█████▎    | 52831/100000 [6:28:04<5:51:14,  2.24it/s]

tensor(589852., grad_fn=<SumBackward0>)


 53%|█████▎    | 53272/100000 [6:31:21<5:43:17,  2.27it/s]


KeyboardInterrupt: 